In [1]:
import numpy as np
import pandas as pd
import sys
import os
from keras.optimizers import Adam
sys.path.append(os.path.join('..'))
import dataloader
from transformer import transformer, extras

Using TensorFlow backend.


In [2]:
X, y, tokens = dataloader.generate_fake_data(11, 7500)
x_tokens = dataloader.TokenList(tokens)
y_tokens = dataloader.TokenList(tokens)
X, y = dataloader.make_data(X, y, x_tokens, y_tokens)

In [3]:
model_size = 512

In [4]:
trans = transformer.Transformer(x_tokens, y_tokens, model_size, 2048, 2, 8)

In [5]:
trans.compile(Adam(0.001, 0.9, 0.98, epsilon=1e-9))

In [6]:
trans.model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
embeddings_1 (Embeddings)       (None, None, 512)    7168        input_1[0][0]                    
__________________________________________________________________________________________________
position_encoding_1 (PositionEn (None, None, 512)    0           embeddings_1[0][0]               
__________________________________________________________________________________________________
dropout_1 (Dropout)             (None, None, 512)    0           position_encoding_1[0][0]        
__________________________________________________________________________________________________
layer_norm

In [7]:
lr_scheduler = extras.LRSchedulerPerStep(model_size, 400)

In [8]:
trans.model.fit([X, y], None, batch_size=30, epochs=1, callbacks=[lr_scheduler])

c:\program files\python36\lib\site-packages\tensorflow\python\ops\gradients_impl.py:112: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/1
7500/7500 [==============================] - 212s 28ms/step - loss: 1.7221 - perplexity: 5.8066 - accu: 0.3420


In [9]:
decoded = trans.decode_sequence([ 1, 2, 3, 4, 5, 6, 7, 8, 9, 10])

In [10]:
print(decoded)

[9, 6, 7, 1, 4, 3, 10, 5, 2, 1, '<EOS>']


In [13]:
X[0], y[0]

(array([ 2,  6,  8,  7,  4, 12,  6,  9, 11,  5, 11,  3]),
 array([ 2,  6,  8,  7,  4, 12,  6,  9, 11,  5, 11,  3]))

In [14]:
x_tokens.id2token

['<PAD>', '<UNK>', '<BOS>', '<EOS>', 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]